In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

In [2]:
input_path = Path('/kaggle/input/br-coins/classification_dataset/all/')
im_size = 320

In [3]:
image_files = list(input_path.glob('*.jpg'))

In [4]:
def read_file(fname):
    # Read image
    im = Image.open(fname)
    im.thumbnail((im_size, im_size))
    im_array = np.asarray(im)# Convert to numpy array
    target = int(fname.stem.split('_')[0])
    return im_array, target

In [5]:
images = []
targets = []

for image_file in tqdm_notebook(image_files):
    image, target = read_file(image_file)
    
    images.append(image)
    targets.append(target)

In [6]:
X = (np.array(images).astype(np.float32) / 127.5) - 1
y_cls = np.array(targets)

In [7]:
X.shape, y_cls.shape

((3059, 240, 320, 3), (3059,))

In [8]:
coins_ids = {
    5: 0,
    10: 1,
    25: 2,
    50: 3,
    100: 4
}

ids_coins = [5, 10, 25, 50, 100]

y = np.array([coins_ids[coin] for coin in y_cls])

In [9]:
X_train, X_valid, y_train, y_valid, fname_train, fname_valid = train_test_split(
    X, y, image_files, test_size=0.2, random_state=42)

In [10]:
im_width = X.shape[2]
im_height = X.shape[1]

im_width, im_height

(320, 240)

In [11]:
from keras.layers import Conv2D, MaxPool2D, Flatten, GlobalAvgPool2D, GlobalMaxPool2D, Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [12]:
model = Sequential()
# CNN network
model.add( Conv2D(16, 3, activation='relu', padding='same', input_shape=(im_height, im_width, 3)) )
model.add( MaxPool2D(2) )

model.add( Conv2D(32, 3, activation='relu', padding='same') )
model.add( MaxPool2D(2) )

model.add( Conv2D(64, 3, activation='relu', padding='same') )
model.add( MaxPool2D(2) )

model.add( Conv2D(128, 3, activation='relu', padding='same') )
model.add( MaxPool2D(2) )

model.add( Conv2D(256, 3, activation='relu', padding='same') )

# Transition between CNN and MLP
model.add( GlobalAvgPool2D() )

# MLP network
model.add( Dense(256, activation='relu') )

model.add( Dense(5, activation='softmax') )

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 240, 320, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 120, 160, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 160, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 60, 80, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 80, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 40, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 40, 128)      

In [13]:
optim = Adam(lr=1e-3)
model.compile(optim, 'sparse_categorical_crossentropy', metrics=['acc'])

In [14]:
callbacks = [
    ReduceLROnPlateau(patience=5, factor=0.1, verbose=True),
    ModelCheckpoint('best.model', save_best_only=True),
    EarlyStopping(patience=12)
]

history = model.fit(X_train, y_train, epochs=2000, validation_data=(X_valid, y_valid), batch_size=32,
                   callbacks=callbacks)

Train on 2447 samples, validate on 612 samples
Epoch 1/2000
2447/2447 [==============================] - 137s 56ms/step - loss: 1.5703 - acc: 0.2456 - val_loss: 1.5799 - val_acc: 0.3350
Epoch 2/2000
2447/2447 [==============================] - 136s 55ms/step - loss: 1.0765 - acc: 0.5513 - val_loss: 1.0381 - val_acc: 0.6095
Epoch 3/2000
2447/2447 [==============================] - 137s 56ms/step - loss: 0.7983 - acc: 0.6951 - val_loss: 1.0966 - val_acc: 0.7663
Epoch 4/2000
2447/2447 [==============================] - 136s 56ms/step - loss: 0.5518 - acc: 0.8120 - val_loss: 0.5886 - val_acc: 0.8775
Epoch 5/2000
2447/2447 [==============================] - 132s 54ms/step - loss: 0.5290 - acc: 0.8128 - val_loss: 0.9110 - val_acc: 0.8235
Epoch 6/2000
2447/2447 [==============================] - 134s 55ms/step - loss: 0.5957 - acc: 0.8194 - val_loss: 0.4769 - val_acc: 0.8578
Epoch 7/2000
2447/2447 [==============================] - 130s 53ms/step - loss: 0.3682 - acc: 0.8864 - val_loss: 0.750

KeyboardInterrupt: 

In [19]:
model.load_weights('best.model')

In [20]:
model.evaluate(X_valid, y_valid)

612/612 [==============================] - 9s 14ms/step


[0.15740982374919008, 0.9673202633857727]

In [21]:
y_pred = model.predict(X_valid)

In [22]:
y_pred_cls = y_pred.argmax(1)

In [23]:
errors = np.where(y_pred_cls != y_valid)[0]
errors

array([  4,  41,  76,  77,  81,  86, 108, 152, 160, 162, 211, 331, 348,
       444, 448, 505, 527, 554, 600, 610])